In [2]:
import pandas as pd

In [2]:
df_basics = pd.read_csv("Data/title.basics.tsv", sep='\t', index_col="tconst")
df_ratings = pd.read_csv("Data/title.ratings.tsv", sep='\t', index_col="tconst")


/var/folders/q5/442qt60n36g3kwnwqzq9_8xw0000gn/T/ipykernel_2860/4095795215.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_basics = pd.read_csv("Data/title.basics.tsv", sep='\t', index_col="tconst")


In [34]:
df_basics2 = df_basics[["titleType", "isAdult", "primaryTitle", "startYear", "runtimeMinutes", "genres"]] # Select only the columns we need
df_basics3 = df_basics2[df_basics2["isAdult"] == 0] # Remove adult movies
df_basics4 = df_basics3[df_basics3["titleType"] == "movie"] # Remove TV shows
df_basics5 = df_basics4.drop(df_basics4[df_basics4['runtimeMinutes'] == "\\N"].index) # Remove movies with no runtime
df_basics6 = df_basics5.drop(df_basics5[df_basics5['genres'] == "\\N"].index) # Remove movies with no genre
df_basics7 = df_basics6.drop(columns=["isAdult", "titleType"]) # Remove isAdult and titleType columns
merged_df = pd.concat([df_basics7, df_ratings], axis=1, join="inner") # Merge the two dataframes
merged_df['genres'] = merged_df['genres'].apply(lambda x: x.strip().split(',')[0]) # Remove all but the first genre of a each movie

In [ ]:
remove_small_genres = merged_df["genres"].value_counts() < 10000 # Remove genres with less than 10000 movies
remove_small_genres2 = remove_small_genres[remove_small_genres == True].index.tolist() # Get the names of the genres to remove
remove_small_genres2.remove('Sci-Fi') # Remove Sci-Fi from the list of genres to remove
merged_df2 = merged_df[~merged_df.genres.isin(remove_small_genres2)] # Remove the small genres

In [15]:
# Normalize the runtime
merged_df2['runtimeMinutes_norm'] = ((merged_df2['runtimeMinutes'] - merged_df2['runtimeMinutes'].mean()) / merged_df2['runtimeMinutes'].std())

# Normalize the rating
merged_df2['averageRating_norm'] = ((merged_df2['averageRating'] - merged_df2['averageRating'].mean()) / merged_df2['averageRating'].std())

# Normalize the popularity
merged_df2['numVotes_norm'] = ((merged_df2['numVotes'] - merged_df2['numVotes'].mean()) / merged_df2['numVotes'].std())

In [24]:
# Export
merged_df2.to_csv("Data/merged_data.csv")

In [26]:
merged_df2

,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,runtimeMinutes_norm,averageRating_norm,numVotes_norm
tconst,,,,,,,,,
tt0000574,The Story of the Kelly Gang,1906,70,Action,6.0,797,-0.165246,-0.128529,-0.090403
tt0000591,The Prodigal Son,1907,90,Drama,5.1,20,-0.033709,-0.803977,-0.110968
tt0000679,The Fairylogue and Radio-Plays,1908,120,Adventure,5.2,66,0.163596,-0.728927,-0.109750
tt0000941,Locura de amor,1909,45,Drama,4.5,23,-0.329668,-1.254275,-0.110889
tt0001184,Don Juan de Serrallonga,1910,58,Adventure,3.9,20,-0.244169,-1.704574,-0.110968
...,...,...,...,...,...,...,...,...,...
tt9916160,Drømmeland,2019,72,Documentary,6.4,49,-0.152093,0.171670,-0.110200
tt9916170,The Rehearsal,2019,51,Drama,7.0,7,-0.290207,0.621969,-0.111312
tt9916190,Safeguard,2020,95,Action,3.7,238,-0.000825,-1.854673,-0.105198
